In [119]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [120]:
from pycaret.utils import version
version()


In [121]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
pd.set_option('max_rows', 90)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from sklearn.neighbors import KNeighborsRegressor
import scipy.stats
from sklearn.preprocessing import StandardScaler
from pycaret.regression import setup, compare_models
from sklearn.model_selection import KFold, cross_val_score

from catboost import CatBoostRegressor
from sklearn.linear_model import BayesianRidge, HuberRegressor, Ridge, OrthogonalMatchingPursuit
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

import optuna

In [122]:
train = pd.read_csv('../input/nyc-taxi-trip-duration/train.zip')

test = pd.read_csv('../input/nyc-taxi-trip-duration/test.zip')

samplesub = pd.read_csv('../input/nyc-taxi-trip-duration/sample_submission.zip')



In [123]:
df=train.copy()

In [124]:
df.head()

In [125]:
df.dtypes

In [126]:
df.isnull().sum()

In [127]:
import re


In [128]:
df['pickmonth'] = pd.to_datetime(df['pickup_datetime']).dt.month
df['pickday'] = pd.to_datetime(df['pickup_datetime']).dt.day
df['pickhr'] = pd.to_datetime(df['pickup_datetime']).dt.hour
df['pickmin'] = pd.to_datetime(df['pickup_datetime']).dt.minute
df['picksec'] = pd.to_datetime(df['pickup_datetime']).dt.second


In [129]:
df['dropmonth'] = pd.to_datetime(df['dropoff_datetime']).dt.month
df['dropday'] = pd.to_datetime(df['dropoff_datetime']).dt.day
df['drophr'] = pd.to_datetime(df['dropoff_datetime']).dt.hour
df['dropmin'] = pd.to_datetime(df['dropoff_datetime']).dt.minute
df['dropsec'] = pd.to_datetime(df['dropoff_datetime']).dt.second

In [130]:
df['dropmonth']

In [131]:
df['pickmonth']

In [132]:
df

In [133]:
df=df.drop('pickup_datetime',axis=1)


In [134]:
df=df.drop('dropoff_datetime',axis=1)

In [135]:
df['id']=df['id'].astype(str)

In [136]:
df['vendor_id'].unique()

In [137]:
df['store_and_fwd_flag'].unique()


In [138]:
df['store_and_fwd_flag'].mask(df['store_and_fwd_flag'] == 'Y', 1, inplace=True)
df['store_and_fwd_flag'].mask(df['store_and_fwd_flag'] == 'N', 0, inplace=True)

In [139]:
df['store_and_fwd_flag']=df['store_and_fwd_flag'].astype(int)

In [140]:
df

In [141]:
df.dtypes

In [142]:
df.columns

In [147]:
y = df['trip_duration']
X = df.drop(['trip_duration','id','dropmonth', 'dropday', 'drophr', 'dropmin',
       'dropsec'],axis=1)

In [148]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [149]:
X = pd.DataFrame(X)

In [150]:
X.columns=['vendor_id', 'passenger_count', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'store_and_fwd_flag',  'pickmonth', 'pickday', 'pickhr',
       'pickmin', 'picksec']

In [151]:
X

In [152]:
test.columns

In [153]:
df1=test.copy()

In [154]:
plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
sns.distplot(y, kde=True, fit=scipy.stats.norm)
plt.title("Without Log Transform")

plt.subplot(1, 2, 2)
sns.distplot(np.log(y), kde=True, fit=scipy.stats.norm)
plt.xlabel("Log trip duration")
plt.title("With Log Transform")

plt.show()

In [155]:
log_y = np.log(y)

In [156]:
df1

In [157]:
df1['pickmonth'] = pd.to_datetime(df1['pickup_datetime']).dt.month
df1['pickday'] = pd.to_datetime(df1['pickup_datetime']).dt.day
df1['pickhr'] = pd.to_datetime(df1['pickup_datetime']).dt.hour
df1['pickmin'] = pd.to_datetime(df1['pickup_datetime']).dt.minute
df1['picksec'] = pd.to_datetime(df1['pickup_datetime']).dt.second


In [158]:
df1=df1.drop('pickup_datetime',axis=1)


In [159]:
df['id']=df['id'].astype(str)

In [160]:
df1['store_and_fwd_flag'].mask(df1['store_and_fwd_flag'] == 'Y', 1, inplace=True)
df1['store_and_fwd_flag'].mask(df1['store_and_fwd_flag'] == 'N', 0, inplace=True)

In [161]:
df1['store_and_fwd_flag']=df1['store_and_fwd_flag'].astype(int)

In [162]:

X1 = df1.drop(['id',],axis=1)

In [163]:
X1.dtypes

In [164]:
X1 = scaler.fit_transform(X1)

In [165]:
X1

In [166]:
X1=pd.DataFrame(X1)

In [167]:
X1.dtypes

In [168]:
X1.columns=['vendor_id', 'passenger_count', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'store_and_fwd_flag',  'pickmonth', 'pickday', 'pickhr',
       'pickmin', 'picksec' ]

In [169]:
X1

In [170]:
!pip install scikit-learn==0.23.2

In [171]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))


In [172]:
X=pd.get_dummies(X)

In [173]:
log_y.shape

In [174]:
 from sklearn.model_selection import train_test_split


In [175]:
X_train, X_rest, y_train, y_rest = train_test_split(X, log_y, train_size=250_000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.8, random_state=200)


In [176]:
X_train.shape

In [177]:
qq = setup(data=pd.concat([X_train,y_train], axis=1), target='trip_duration')

In [178]:
compare_models()

In [179]:

import optuna
import xgboost as xgb
from sklearn.metrics import r2_score

In [181]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_rest, label=y_rest)
dval = xgb.DMatrix(X_val, label=y_val)



In [182]:
dtest1=xgb.DMatrix(X1)

In [183]:
def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 0.00001, 10.0)
    max_depth = trial.suggest_int('max_depth', 4, 8)
    l1_reg = trial.suggest_loguniform('l1_reg', 0.00001, 10.0)
    l2_reg = trial.suggest_loguniform('l2_reg', 0.00001, 10.0)
    
    params = {'learning_rate': learning_rate, 'max_depth': max_depth, 'alpha': l1_reg, 'lambda': l2_reg}
    
    return get_model_rmse(params)


In [184]:
def get_model_rmse(params):
    model = xgb.train(params, dtrain, num_boost_round=100, evals=[(dval, 'eval')], early_stopping_rounds=10, verbose_eval=0)
    results = model.eval(dval)
    rmse = np.float(re.search(r'[\d.]+$', results).group(0))
    return rmse

In [186]:
study = optuna.create_study()
study.optimize(objective, n_trials=100, show_progress_bar=True)

In [187]:
best_params = study.best_params
best_params

In [188]:
model = xgb.train(best_params, dtrain, num_boost_round=10000, evals=[(dval, 'eval')], early_stopping_rounds=10)


In [189]:
y_pred = np.array(model.predict(dtest1), dtype=np.float)



In [190]:
y_pred

In [196]:
y_pred['id']=test['id']

In [191]:
y_pred=pd.DataFrame(y_pred)

In [202]:
y_pred.columns

In [203]:
y_pred['trip_duration']=y_pred[0]

In [207]:
y_pred1=y_pred.drop(y_pred.columns[[0,2]],axis=1)

In [209]:
y_pred1['trip_duration']=np.exp(y_pred1['trip_duration'])

In [211]:
y_pred1

In [213]:
y_pred1['trip_duration'].round(0)

In [214]:
y_pred1['trip_duration']=y_pred1['trip_duration'].astype(int)

In [215]:
y_pred1

In [216]:
y_pred1.to_csv('./submission.csv', index=False, header=True)